In [1]:
import numpy as np  
m = 100 
t = 0 
s0 = 100       
sigma=0.08  
rho=0.02 
T = 1 
K=100 
N = 100 
def f(a,b,n,K,x): 
    index = np.arange(n+1)
    c = K-(x*(1+a)**index * (1+b)**(n-index))
    return np.where(c>0,c,0)

def Fr(t,x,rho,sigma,T,K,m):
    cte = (sigma*np.sqrt(T) /np.sqrt(m))  
    b = np.exp(cte) - 1 
    a = np.exp(-cte) - 1 
    r = rho*T / m
    n = round(m*T)- round(m*t) 
    V = np.zeros((n+1,n+1))
    V[:,n] = f(a,b,n,K,x)
    p  = (b-r) / (b-a) 
    for i in range(n-1,-1,-1): 
        S  = (1/(1+r))*((1-p)*V[0:i+1,i+1] + p*V[1:i+2,i+1])
        M = np.c_[S,f(a,b,i,K,x)] 
        V[0:i+1,i] = np.max(M,axis = 1)
    return V[0,0] 
    
print(Fr(t,s0,rho,sigma,T,K,m))


2.4567334867990303


In [2]:
m = 100
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt 
from matplotlib import cm 
N = 100 
fig = plt.figure()
ax = fig.gca(projection='3d')
lt= np.linspace(T/N,T-T/N,N-2)
Yt = np.linspace(0,20,N)
ltm, Ym = np.meshgrid(lt, Yt)
Vt = [[Fr(float(t),x,rho,sigma,T,K,m) for t in lt] for x in Yt] 
Vt = np.array(Vt)
ax.plot_surface(ltm,Ym, Vt, cmap=cm.viridis)   
plt.show() 

<Figure size 640x480 with 1 Axes>

In [3]:
M= 250     
N=1000
KI=np.log(2*K) 
T=1 
sigma=2 
rho=3 
K = 10 
lt= np.linspace(0,T,N) 
dt=T/N
dy= (2*KI/M)*np.linspace(0,M,M)
dx=(2*K/M)
Y= -KI*np.ones((1,M))+dy  


A  = 2*np.eye(M) - np.eye(M, k=1) - np.eye(M, k=-1)  
A[0,0:3]=[-1,2,-1]
A[M-1,M-3:M]=[-1,2,-1]
D=np.eye(M)-np.eye(M, k=1) 
V=np.zeros((M,N))
f = lambda x: max(K-np.exp(x),0) 
f = np.vectorize(f) 
V[:,N-1]=f(Y)
MM=np.eye(M)*(1-rho*dt)+dt*(rho-(sigma**2/2))/dx*D-A*(sigma**2)/2*dt/dx**2 
MM[0,:]=np.zeros((1,M))
MM[M-1,:]=np.zeros((1,M))  
for i in range((N-2),-1,-1):
    V[:,i]= np.maximum(np.dot(MM,V[:,i+1]), f(Y))


In [20]:
 
index = int((M/2)+(M*np.log(s0))/(2*KI)) 
print(index) 
V[index,0]

216


1.340185954520961

In [21]:
print(Y[0,index])  
print("----------")  
print(np.log(s0))

2.201682755262571
----------
2.1972245773362196


In [38]:
import numpy as np
def OptioncallE(s0,Smax,M,T,N,K,r,sigma):
    
    ds=Smax/float(M)   
    dt=T/float(N)      
    
    i=np.arange(1,M,dtype=np.float)   
    
    
    P=(-.5*r*dt*i+.5*sigma**2*dt*i**2)/(1+r*dt)
    Q=(1-sigma**2*dt*i**2)/(1+r*dt)
    R=(.5*r*dt*i+.5*sigma**2*dt*i**2)/(1+r*dt)
    
    A=np.diag(Q)+np.diag(P[1:],k=-1)+np.diag(R[0:M-2],k=1)
    
    F=np.zeros((N+1,M+1)) 
    
    F[:,0]=0
    F[:,M]=[Smax * np.exp(-r*( N - j)*dt) for j in range(N+1)]
    F[N,:]=np.maximum(K-np.arange(0,Smax+ds/2.0,ds,dtype=np.float),0)
    
    F=np.matrix(np.array(F))
    
   
    for j in range(N-1,-1,-1): 
        d=np.zeros((M-1,1))
        
        d[0]=P[0]*F[j+1,0] 
        d[M-2]=R[M-2]*F[j+1,M]
        
        F[j,1:M]=np.dot(F[j+1,1:M],A)+np.matrix(np.array(d)).transpose() #computation
        F[j,:]=np.maximum(K-np.arange(0,Smax+ds/2.0,ds,dtype=np.float),F[j,:]) 
    return F[0,int(s0/ds)]
    
        
print("The value of an American call option is" , OptioncallE(s0,20,50,T,150,K,rho,sigma))                  

The value of an American call option is 1.1999999999999993
